# Part 1: Creating and rendering an environment


In this notebook, we will see how to create, interact with and render our first railway systems.

# In a nutshell

In [ ]:
import numpy as np

from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import rail_from_manual_specifications_generator

In [ ]:
# Fixed railway: hardcoded 2D tuple of (tile type, rotation)
specs = [[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
         [(0, 0), (0, 0), (0, 0), (0, 0), (7, 0), (0, 0)],
         [(7, 270), (1, 90), (1, 90), (1, 90), (2, 90), (7, 90)],
         [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]]

rail_shape = np.array(specs).shape

fixed_env = RailEnv(width=rail_shape[1],
              height=rail_shape[0],
              rail_generator=rail_from_manual_specifications_generator(specs),
              number_of_agents=1
              )

# Call reset() to initialize the env
observation = fixed_env.reset()

In [ ]:
"""
For each agent, the observation indicates:
- the transition map array
- the state of all agents in the environment
- the position and targets of all the agents

We will see how to derive and use more useful observations in the next part.
"""
observation

In [ ]:
import PIL
from flatland.utils.rendertools import RenderTool

# Render it
def render_env(env):
    env_renderer = RenderTool(env, gl="TKPIL")
    env_renderer.render_env()

    image = env_renderer.get_image()
    pil_image = PIL.Image.fromarray(image)
    display(pil_image)

render_env(fixed_env)

In [ ]:
from flatland.envs.rail_generators import random_rail_generator

# Random railway: built from list of probability per cell type
transition_probability = [1.0,  # empty cell - Case 0
                          1.0,  # Case 1 - straight
                          1.0,  # Case 2 - simple switch
                          0.3,  # Case 3 - diamond drossing
                          0.5,  # Case 4 - single slip
                          0.5,  # Case 5 - double slip
                          0.2,  # Case 6 - symmetrical
                          0.0,  # Case 7 - dead end
                          0.2,  # Case 8 - turn left
                          0.2,  # Case 9 - turn right
                          1.0]  # Case 10 - mirrored switch

random_env = RailEnv(width=15,
              height=15,
              rail_generator=random_rail_generator(
                        cell_type_relative_proportion=transition_probability
                        ),
              number_of_agents=1)

random_env.reset();

In [ ]:
render_env(random_env)

# In more details

Let's go through the code above step by step.

Creating a rail network
---

There are multiple ways to generate a rail network. The simpler one is to describe it explicitely, as such:

In [ ]:
specs = [[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
         [(0, 0), (0, 0), (0, 0), (0, 0), (7, 0), (0, 0)],
         [(7, 270), (1, 90), (1, 90), (1, 90), (2, 90), (7, 90)],
         [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]]

`specs` is a 2-dimensional array of tuples:

In [ ]:
import numpy as np

rail_shape = np.array(specs).shape
rail_shape

The `specs` array represent a 4 by 6 2D grid of tuples. In each tuple, the first element represent the **cell type**, and the second the **rotation** of the cell (0, 90, 180 or 270 degrees clockwise).

Check `flatland.core.grid.rail_env_grid.RailEnvTransitions` for implementation details.

In [ ]:
env = RailEnv(width=rail_shape[1],
              height=rail_shape[0],
              rail_generator=rail_from_manual_specifications_generator(specs),
              number_of_agents=1
              )

A call to `reset()` is necessary to fully initialize the environment.

In [ ]:
observation = env.reset()

np.array(observation).shape

As usual with gym-like environment, it returns the initial observation as a results.

In [ ]:
observation

**The observation gives a global overview of the entire rail environment.**

It is composed of the following elements:
- a transition map array with dimensions (env.height, env.width, 16)
- obs_agents_state: A 3D array (map_height, map_width, 5) with
    - first channel containing the agents position and direction
    - second channel containing the other agents positions and direction
    - third channel containing agent/other agent malfunctions
    - fourth channel containing agent/other agent fractional speeds
    - fifth channel containing number of other agents ready to depart
- obs_targets: Two 2D arrays (map_height, map_width, 2) containing respectively the position of the given agent target and the positions of the other agents targets (flag only, no counter!).

**That's it!** You have just created your first rail environment. Let's render it to see how it looks.

Rendering the environment
---

In [ ]:
import PIL

env_renderer = RenderTool(env, gl="TKPIL")
env_renderer.render_env()

In [ ]:
arrImage = env_renderer.get_image()
pilImage = PIL.Image.fromarray(arrImage)
display(pilImage)

Creating a random environment
---

Alternatively, a random environment can be generated (optionally specifying weights for each cell type to increase or decrease their proportion in the generated rail networks).



In [ ]:
# Random railway: built from list of probability per cell type
transition_probability = [1.0,  # empty cell - Case 0
                          1.0,  # Case 1 - straight
                          1.0,  # Case 2 - simple switch
                          0.3,  # Case 3 - diamond drossing
                          0.5,  # Case 4 - single slip
                          0.5,  # Case 5 - double slip
                          0.2,  # Case 6 - symmetrical
                          0.0,  # Case 7 - dead end
                          0.2,  # Case 8 - turn left
                          0.2,  # Case 9 - turn right
                          1.0]  # Case 10 - mirrored switch

random_env = RailEnv(width=15,
              height=15,
              rail_generator=random_rail_generator(
                        cell_type_relative_proportion=transition_probability
                        ),
              number_of_agents=1)

random_env.reset();

Animated rendering
---

We will use [ipycanvas](https://github.com/martinRenou/ipycanvas) to cleanly display the environment in the notebook.
Note that Flatland comes with multiple rendering options.

In [ ]:
from ipycanvas import Canvas

In [ ]:
env_renderer = render_pil = RenderTool(env, gl="PILSVG",
                                       agent_render_variant=AgentRenderVariant.ONE_STEP_BEHIND,
                                       show_debug=False,
                                       screen_height=1000,  # Adjust these parameters to fit your resolution
                                       screen_width=1300)  # Adjust these parameters to fit your resolution

env_renderer.reset()

In [ ]:
render_pil.render_env(show=False, show_observations=False, show_predictions=False, show_agents=True)
img = render_pil.get_image()

canvas = Canvas(size=(img.shape[0], img.shape[1]))
canvas.put_image_data(img)

In [ ]:
canvas